# Run simple simulation
## Load module

In [ ]:
import PyRAMSES

## Load simulator

In [ ]:
ram = PyRAMSES.sim()

## Load saved test-case

In [ ]:
case = PyRAMSES.cfg('cmd.txt')

## Add one observation more

In [ ]:
case.addRunObs('MS g2') # will plot in real-time the voltage on bus g1

## Run simulation and pause at t=0 seconds

In [ ]:
ram.execSim(case,0.0)

### 1. Set P-term, I-term

In [ ]:
comp_type = ['SYN']
comp_name = ['g2']
obs_name = ['Omega']

kp = 0.2
ki = 0.1
errSum = 0
t=300
nominal_frequency = 1.0

### 2. Get delta frequency (error)

In [ ]:
for i in range(1,t):
    actual_frequency = ram.getObs(comp_type,comp_name, obs_name)[0]
    error = nominal_frequency - actual_frequency
    print(error)
    errSum += error*1.0
    print(errSum)
    output = kp * error + ki * errSum
    # loop to send measurements to gens
    
    ram.contSim(i)

## Split output into 6 generators

In [ ]:
#Tm0 = output/6


## Pass these setpoints to the generators g13-g18

In [ ]:
list_of_gens = ['g13', 'g14', 'g15', 'g16', 'g17', 'g18']

for gen in list_of_gens:
    command = 'CHGPRM TOR' + gen + 'Tm0' + str(output/6) + '0'
    ram.addDisturb(ram.getSimTime()+0.01, command)

## Simulate to the end

In [ ]:
ram.contSim(ram.getInfTime())